<a href="https://colab.research.google.com/github/Docas32/Potencial-energetico-solar/blob/main/teste_002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install earthengine-api pandas numpy geopandas

In [ ]:
import ee
import pandas as pd
import geopandas as gpd
import re
from unicodedata import normalize

In [ ]:
# Inicializa o Earth Engine
try:
    ee.Initialize()
    print("Google Earth Engine inicializado com sucesso.")
except Exception as e:
    print(f"Erro ao inicializar GEE: {e}")
# Carregue seus limites municipais do Paraná (Shapefile ou GeoJSON)
municipios_gdf = gpd.read_file("/content/drive/MyDrive/PR_Municipios_2024/PR_Municipios_2024.shp")

Google Earth Engine inicializado com sucesso.


In [ ]:
df_hsp = pd.read_csv('/content/drive/MyDrive/global_horizontal_means_PR_sedes-munic.csv', delimiter=';')
# 1. Encontrar o mínimo e máximo para HSP
X_min_hsp = df_hsp['ANNUAL'].min()
X_max_hsp = df_hsp['ANNUAL'].max()

# 2. Aplicar a normalização
df_hsp['Z_HSP'] = (df_hsp['ANNUAL'] - X_min_hsp) / (X_max_hsp - X_min_hsp)

# Verificação:
print(f"HSP Mínimo (X_min): {X_min_hsp}")
print(f"HSP Máximo (X_max): {X_max_hsp}")
print(f"Z_HSP Mínimo (deve ser 0): {df_hsp['Z_HSP'].min()}")
print(f"Z_HSP Máximo (deve ser 1): {df_hsp['Z_HSP'].max()}")

HSP Mínimo (X_min): 3785
HSP Máximo (X_max): 5062
Z_HSP Mínimo (deve ser 0): 0.0
Z_HSP Máximo (deve ser 1): 1.0


In [ ]:
municipios_ee = ee.FeatureCollection('projects/neon-equinox-428813-h8/assets/limites_pr_2024')
print("Limites municipais carregados diretamente do Asset GEE.")

Limites municipais carregados diretamente do Asset GEE.


In [ ]:
# @title
# Carregar o MDE (SRTM 30m)
srtm = ee.Image('USGS/SRTMGL1_003').select('elevation')

# Calcular a Declividade (Slope) em graus
declividade = ee.Terrain.slope(srtm)

# Função para extrair a média zonal por município
stats_declividade = declividade.reduceRegions(
    collection=municipios_ee,
    reducer=ee.Reducer.mean(),
    scale=30 # Resolução do SRTM
)

# Converter o resultado para uma lista de dicionários Python
X_D_list = stats_declividade.getInfo()['features']

In [ ]:
# 1. Carregar a Image Collection 8 completa (caminho público oficial)
mapbiomas_collection = ee.ImageCollection(
    'projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1'
)

# 2. Filtrar o ano desejado (2022) e obter a imagem única
mapbiomas_2022 = mapbiomas_collection.filterMetadata(
    'year', 'equals', 2022
).first()

# 3. Selecionar a banda de classificação
classes = mapbiomas_2022.select('classification')

In [ ]:
# @title
# =================================================================================
# CÁLCULO DO FATOR DE SOMBREAMENTO (X_S)
# =================================================================================

print("1. Calculando Hillshade (proxy para sombreamento)...")

# Reutiliza o MDE (srtm) que contém a elevação
srtm = ee.Image('USGS/SRTMGL1_003').select('elevation')

# 1. Calcular Hillshade (Simulação de Sombra e Iluminação)
# Usamos o padrão 45 graus para Azimute e Ângulo Solar.
hillshade = ee.Terrain.hillshade(srtm, 45, 45) # Azimute e Ângulo

# 2. Inversão para Fator de Sombreamento (X_S)
# Hillshade é 0 (sombra total) a 255 (luz total).
# Fator Sombra (0 = luz, 1 = sombra):
# Fator Sombra = (255 - Hillshade) / 255
fator_sombreamento = ee.Image.constant(255).subtract(hillshade).divide(255).rename('fator_sombra')


# 3. AGREGAÇÃO ZONAL (Cálculo da Média de Sombreamento)
print("2. Agregando fator de sombreamento (X_S) por município...")

stats_sombreamento = fator_sombreamento.reduceRegions(
    collection=municipios_ee,
    reducer=ee.Reducer.mean(),
    scale=30 # Resolução de processamento
)

# 4. CONVERSÃO PARA LISTA PYTHON
# -----------------------------------------------------------------------
X_S_list = stats_sombreamento.getInfo()['features']
print("Cálculo de X_S concluído. Dados prontos para conversão em DataFrame.")

1. Calculando Hillshade (proxy para sombreamento)...
2. Agregando fator de sombreamento (X_S) por município...
Cálculo de X_S concluído. Dados prontos para conversão em DataFrame.


In [ ]:
# FUNÇÃO DE PADRONIZAÇÃO ROBUSTA
def padronizar_nome(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto).upper()
    texto = normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    texto = re.sub(r'[^A-Z0-9]', '', texto)
    return texto

# --- CARREGAMENTO E PADRONIZAÇÃO ---

# 1. Carregar Shapefile do IBGE corrigindo a codificação
caminho_shp = "/content/drive/MyDrive/PR_Municipios_2024/PR_Municipios_2024.shp"
municipios_gdf = gpd.read_file(caminho_shp, encoding='cp1252')
municipios_gdf = municipios_gdf[municipios_gdf['SIGLA_UF'] == 'PR'].copy()

# 2. Criar DF do IBGE e padronizar o nome
df_ibge = pd.DataFrame(municipios_gdf[['NM_MUN', 'CD_MUN', 'SIGLA_UF']])
df_ibge['NOME_LIMPO'] = df_ibge['NM_MUN'].apply(padronizar_nome)

# 3. Carregar DF de HSP (Assumindo que df_hsp já está carregado)
# df_hsp = pd.read_csv('caminho/para/seus/hsp_data.csv')
df_hsp['NOME_LIMPO'] = df_hsp['NAME'].apply(padronizar_nome)

# 4. CRIAR O DF DE MAPEAMENTO (DF_MAPA) - Incluindo a coluna 'NAME'
df_mapa = df_hsp[['ID', 'NAME', 'NOME_LIMPO', 'ANNUAL']].merge(
    df_ibge[['CD_MUN', 'NOME_LIMPO']],
    on='NOME_LIMPO',
    how='inner'
)
# O df_mapa agora contém 'ID', 'NAME', 'ANNUAL' (X_HSP) e o 'CD_MUN'.
# df_mapa agora contém 'ID', 'ANNUAL' (X_HSP) e o 'CD_MUN' correto.
df_mapa = df_mapa.rename(columns={'ANNUAL': 'X_HSP_HSP_Anual'})
print(f"Mapeamento criado com {len(df_mapa)} municípios.")

# --- Variáveis GEE (Assumindo que X_D_list e X_S_list foram calculadas e convertidas) ---

# Função GEE para DF (utiliza CD_MUN como chave)
def gee_to_df(gee_list, new_col_name, id_col_name='CD_MUN'):
    data = []
    for f in gee_list:
        municipio_id = str(f['properties'][id_col_name])
        result_value = f['properties']['mean']
        data.append({'CD_MUN': municipio_id, new_col_name: result_value})
    return pd.DataFrame(data)

df_xd = gee_to_df(X_D_list, 'X_D_Declividade_Media', id_col_name='CD_MUN')
df_xs = gee_to_df(X_S_list, 'X_S_Fator_Sombreamento', id_col_name='CD_MUN')

Mapeamento criado com 399 municípios.


In [ ]:
# @title
# 5. MERGE FINAL DOS DADOS (X_HSP, X_D, X_S)
# O merge usa a chave CD_MUN (Código IBGE)
df_final = df_mapa.merge(df_xd, on='CD_MUN', how='inner')
df_final = df_final.merge(df_xs, on='CD_MUN', how='inner')

# 6. NORMALIZAÇÃO MIN-MAX (Escala 0 a 1)
COLS_VARIAVEIS = ['X_HSP_HSP_Anual', 'X_D_Declividade_Media', 'X_S_Fator_Sombreamento']

for col in COLS_VARIAVEIS:
    X_min = df_final[col].min()
    X_max = df_final[col].max()

    if 'HSP' in col:
        # Positivo (maior é melhor)
        df_final[f'Z_{col}'] = (df_final[col] - X_min) / (X_max - X_min)

    elif 'Declividade' in col or 'Sombreamento' in col:
        # Negativo (menor é melhor)
        df_final[f'Z_{col}'] = (X_max - df_final[col]) / (X_max - X_min)

# 7. PONDERAÇÃO E CÁLCULO DO SCORE FINAL
W_HSP = 0.35
W_S   = 0.30
W_D   = 0.35

df_final['Score_Potencial_Bruto'] = (
    (df_final['Z_X_HSP_HSP_Anual'] * W_HSP) +
    (df_final['Z_X_S_Fator_Sombreamento'] * W_S) +
    (df_final['Z_X_D_Declividade_Media'] * W_D)
)

# Escalar para 0 a 100
df_final['Score_Final_Sitio'] = df_final['Score_Potencial_Bruto'] * 100

print("\nCálculo do Score de Potencial Bruto (0-100) concluído.")
print("Municípios com maior Potencial (Top 5):")
print(df_final[['NAME', 'Score_Final_Sitio']].sort_values(by='Score_Final_Sitio', ascending=False).head())


Cálculo do Score de Potencial Bruto (0-100) concluído.
Municípios com maior Potencial (Top 5):
                       NAME  Score_Final_Sitio
396           Jardim Olinda          85.538583
393  Santo Antônio do Caiuá          84.857988
390       Diamante do Norte          84.793293
381           Nova Londrina          83.570838
392              Terra Rica          83.552324


In [ ]:


# A função padronizar_nome deve estar definida no seu script.

def consultar_score(df_resultado, nome_municipio=None):
    """
    Filtra o DataFrame, exibe as variáveis BRUTAS, o Fator de Capacidade e o Score Final.
    """

    # 1. Aplicar Cálculo do Fator de Capacidade (FC) - Adicionamos ao DF para o cálculo
    PR_PADRAO = 0.80 # 80% de Performance Ratio (estimativa conservadora de perdas)
    HORAS_NO_ANO = 8760

        # Correção: Divide a irradiação por 1000 (para converter Wh/m²/dia para kWh/m²/dia, que é o HSP)
    df_resultado['HSP_CORRIGIDO'] = df_resultado['X_HSP_HSP_Anual'] / 1000

    PR_PADRAO = 0.80
    HORAS_NO_ANO = 8760

    # Usa o valor corrigido para o cálculo do FC
    df_resultado['Fator_Capacidade_Est'] = (
        (df_resultado['HSP_CORRIGIDO'] * 365 / HORAS_NO_ANO) * PR_PADRAO
    ) * 100 # Multiplica por 100 para ser em %


    # 2. Aplicar Filtro
    if nome_municipio:
        nome_padronizado = padronizar_nome(nome_municipio)
        df_resultado['NAME_FILTRO'] = df_resultado['NAME'].apply(padronizar_nome)

        df_filtrado = df_resultado[
            df_resultado['NAME_FILTRO'].str.contains(nome_padronizado)
        ].copy()

        df_filtrado = df_filtrado.drop(columns=['NAME_FILTRO'])

        if df_filtrado.empty:
            print(f"Nenhum município encontrado com o nome '{nome_municipio}'.")
            return
    else:
        # Se não houver filtro, mostra o ranking completo (Top 10)
        df_filtrado = df_resultado.sort_values(by='Score_Final_Sitio', ascending=False).head(10).copy()
        print("\n--- TOP 10 MUNICÍPIOS POR SCORE DE POTENCIAL BRUTO ---")


    # 3. Exibir Resultados (Valores Brutos e Score Final)

    COLUNAS_BRUTAS_EXIBICAO = [
        'NAME',
        'X_HSP_HSP_Anual',
        'Fator_Capacidade_Est',
        'X_D_Declividade_Media',
        'X_S_Fator_Sombreamento',
        'Score_Final_Sitio'
    ]

    print("\n-------------------------------------------------------------------------------------")
    print("      ANÁLISE DO POTENCIAL BRUTO E VIABILIDADE TÉCNICA (FASE 1)      ")
    print("-------------------------------------------------------------------------------------")

    df_exibicao = df_filtrado[COLUNAS_BRUTAS_EXIBICAO].rename(columns={
        'NAME': 'Município',
        'X_HSP_HSP_Anual': 'HSP (Horas/Dia)',
        'Fator_Capacidade_Est': 'Fator de Capacidade Est. (%)',
        'X_D_Declividade_Media': 'Declividade Média (°)',
        'X_S_Fator_Sombreamento': 'Fator Sombreamento (0-1)'
    })

    # Formatação das colunas numéricas
    df_exibicao = df_exibicao.round({
        'HSP (Horas/Dia)': 2,
        'Fator de Capacidade Est. (%)': 2,
        'Declividade Média (°)': 2,
        'Fator Sombreamento (0-1)': 3,
        'Score_Final_Sitio': 1
    })

    print(df_exibicao.to_markdown(index=False))
    print("-------------------------------------------------------------------------------------")

# Exemplo de uso:

# 1. Consultar Maringá
consultar_score(df_final, nome_municipio="Cerro Azul")

# 2. Mostrar Top 10
# consultar_score(df_final)


-------------------------------------------------------------------------------------
      ANÁLISE DO POTENCIAL BRUTO E VIABILIDADE TÉCNICA (FASE 1)      
-------------------------------------------------------------------------------------
| Município   |   HSP (Horas/Dia) |   Fator de Capacidade Est. (%) |   Declividade Média (°) |   Fator Sombreamento (0-1) |   Score_Final_Sitio |
|:------------|------------------:|-------------------------------:|------------------------:|---------------------------:|--------------------:|
| Cerro Azul  |              4223 |                          14.08 |                   17.24 |                      0.296 |                  31 |
-------------------------------------------------------------------------------------


In [ ]:
# --- FUNÇÃO AUXILIAR: EXIBIR CRITÉRIOS ---
def exibir_criterios_de_analise():
    """Exibe a tabela de critérios de referência para as variáveis brutas analisadas."""
    print("\n-------------------------------------------------------------------------------------")
    print("                      CRITÉRIOS DE REFERÊNCIA PARA VARIÁVEIS BRUTAS                      ")
    print("-------------------------------------------------------------------------------------")

    # Critérios Baseados na realidade do Paraná e melhores práticas de projetos fotovoltaicos
    criterios = {
        'HSP (Horas de Sol Pleno - H/Dia)': [
            ('> 4.8 h/dia', 'Ótimo (Excelente Potencial)'),
            ('4.3 h/dia a 4.8 h/dia', 'Bom (Potencial Sólido)'),
            ('< 4.3 h/dia', 'Razoável/Ruim (Baixo Potencial)')
        ],
        'Declividade Média (°)' : [
            ('< 5°', 'Ótimo (Requer menor terraplanagem)'),
            ('5° a 10°', 'Aceitável (Pode exigir ajustes)'),
            ('> 10°', 'Ruim (Alto custo de implantação)')
        ],
        'Fator Sombreamento (0-1)' : [
            ('> 0.95', 'Ótimo (Pouca ou nenhuma obstrução)'),
            ('0.90 a 0.95', 'Razoável (Sombreamento leve)'),
            ('< 0.90', 'Ruim (Risco significativo de perda de geração)')
        ]
    }

    # Criando e exibindo a tabela
    df_criterios = pd.DataFrame([
        {'Variável': 'HSP (Horas/Dia)', 'Faixa': c[0], 'Interpretação': c[1]} for c in criterios['HSP (Horas de Sol Pleno - H/Dia)']
    ] + [
        {'Variável': 'Declividade Média (°)', 'Faixa': c[0], 'Interpretação': c[1]} for c in criterios['Declividade Média (°)']
    ] + [
        {'Variável': 'Fator Sombreamento (0-1)', 'Faixa': c[0], 'Interpretação': c[1]} for c in criterios['Fator Sombreamento (0-1)']
    ])

    print(df_criterios.to_markdown(index=False))
    print("-------------------------------------------------------------------------------------\n")


def consultar_score(df_resultado, nome_municipio=None):
    """
    Filtra o DataFrame, exibe as variáveis BRUTAS (corrigidas), o Fator de Capacidade e o Score Final.
    """

    # 1. ANÁLISE E CORREÇÃO DAS UNIDADES (Fase Crítica)
    PR_PADRAO = 0.80
    HORAS_NO_ANO = 8760

    # CONVERSÃO: Wh/m²/dia para Horas de Sol Pleno (HSP em horas/dia)
    df_resultado['HSP_CORRIGIDO_H/DIA'] = df_resultado['X_HSP_HSP_Anual'] / 1000

    # CÁLCULO DO FATOR DE CAPACIDADE (FC)
    df_resultado['Fator_Capacidade_Est'] = (
        (df_resultado['HSP_CORRIGIDO_H/DIA'] * 365 / HORAS_NO_ANO) * PR_PADRAO
    ) * 100

    # 2. Aplicar Filtro
    if nome_municipio:
        nome_padronizado = padronizar_nome(nome_municipio)
        df_resultado['NAME_FILTRO'] = df_resultado['NAME'].apply(padronizar_nome)

        df_filtrado = df_resultado[
            df_resultado['NAME_FILTRO'].str.contains(nome_padronizado)
        ].copy()

        df_filtrado = df_filtrado.drop(columns=['NAME_FILTRO'])

        if df_filtrado.empty:
            print(f"Nenhum município encontrado com o nome '{nome_municipio}'.")
            return
    else:
        # Se não houver filtro, mostra o ranking completo (Top 10)
        df_filtrado = df_resultado.sort_values(by='Score_Final_Sitio', ascending=False).head(10).copy()
        print("\n--- TOP 10 MUNICÍPIOS POR SCORE DE POTENCIAL BRUTO ---")


    # 3. Exibir Resultados (Valores Brutos e Score Final)

    COLUNAS_BRUTAS_EXIBICAO = [
        'NAME',
        'HSP_CORRIGIDO_H/DIA',
        'Fator_Capacidade_Est',
        'X_D_Declividade_Media',
        'X_S_Fator_Sombreamento',
        'Score_Final_Sitio'
    ]

    print("\n-------------------------------------------------------------------------------------")
    print("      RESULTADO DA ANÁLISE TÉCNICA (FASE 1)      ")
    print("-------------------------------------------------------------------------------------")

    df_exibicao = df_filtrado[COLUNAS_BRUTAS_EXIBICAO].rename(columns={
        'NAME': 'Município',
        'HSP_CORRIGIDO_H/DIA': 'HSP (Horas/Dia)',
        'Fator_Capacidade_Est': 'Fator de Capacidade Est. (%)',
        'X_D_Declividade_Media': 'Declividade Média (°)',
        'X_S_Fator_Sombreamento': 'Fator Sombreamento (0-1)'
    })

    # Formatação das colunas numéricas
    df_exibicao = df_exibicao.round({
        'HSP (Horas/Dia)': 2,
        'Fator de Capacidade Est. (%)': 2,
        'Declividade Média (°)': 2,
        'Fator Sombreamento (0-1)': 3,
        'Score_Final_Sitio': 1
    })

    print(df_exibicao.to_markdown(index=False))
    print("-------------------------------------------------------------------------------------")

# --- USO DA FUNÇÃO: INCLUINDO OS CRITÉRIOS ANTES DA CONSULTA ---

# 1. Exibir a tabela de critérios de análise
exibir_criterios_de_analise()

# 2. Consultar um município específico:
# consultar_score(df_final, nome_municipio="Maringa")

# 3. Mostrar o ranking completo (Top 10):
# consultar_score(df_final)


-------------------------------------------------------------------------------------
                      CRITÉRIOS DE REFERÊNCIA PARA VARIÁVEIS BRUTAS                      
-------------------------------------------------------------------------------------
| Variável                 | Faixa                 | Interpretação                                  |
|:-------------------------|:----------------------|:-----------------------------------------------|
| HSP (Horas/Dia)          | > 4.8 h/dia           | Ótimo (Excelente Potencial)                    |
| HSP (Horas/Dia)          | 4.3 h/dia a 4.8 h/dia | Bom (Potencial Sólido)                         |
| HSP (Horas/Dia)          | < 4.3 h/dia           | Razoável/Ruim (Baixo Potencial)                |
| Declividade Média (°)    | < 5°                  | Ótimo (Requer menor terraplanagem)             |
| Declividade Média (°)    | 5° a 10°              | Aceitável (Pode exigir ajustes)                |
| Declividade Média (°)

In [ ]:
# --- CÁLCULO PERMANENTE DO HSP  E FATOR DE CAPACIDADE ---

PR_PADRAO = 0.80
HORAS_NO_ANO = 8760

# 1. CORREÇÃO DA UNIDADE: Converte Wh/m²/dia (X_HSP_HSP_Anual) para Horas de Sol Pleno (HSP em horas/dia)
# Isso corrige o erro de escala (16300%)
df_final['HSP_CORRIGIDO_H/DIA'] = df_final['X_HSP_HSP_Anual'] / 1000

# 2. CÁLCULO PERMANENTE DO FATOR DE CAPACIDADE (FC)
df_final['Fator_Capacidade_Est'] = (
    (df_final['HSP_CORRIGIDO_H/DIA'] * 365 / HORAS_NO_ANO) * PR_PADRAO
) * 100 # Em porcentagem

print("Cálculos de Unidade (HSP Corrigido e Fator de Capacidade) finalizados e adicionados ao DataFrame.")

# O restante do seu script de relatório (definição de COLUNAS_RELATORIO, ordenação e exportação) pode ser executado agora sem o KeyError.

Cálculos de Unidade (HSP Corrigido e Fator de Capacidade) finalizados e adicionados ao DataFrame.


In [ ]:
# ['NAME', 'X_HSP_HSP_Anual', 'X_D_Declividade_Media', 'X_S_Fator_Sombreamento', 'Score_Final_Sitio']
# E as colunas calculadas: 'HSP_CORRIGIDO_H/DIA' e 'Fator_Capacidade_Est'.

# 1. Definir as colunas para o relatório final
COLUNAS_RELATORIO = [
    'NAME',
    'HSP_CORRIGIDO_H/DIA',
    'X_D_Declividade_Media',
    'X_S_Fator_Sombreamento',
    'Fator_Capacidade_Est',
    'Score_Final_Sitio'
]

# 2. Criar uma cópia do DF para o relatório e adicionar o Ranking
df_relatorio = df_final[COLUNAS_RELATORIO].copy()

# Classificar pelo Score e adicionar o ranking
df_relatorio = df_relatorio.sort_values(by='Score_Final_Sitio', ascending=False)
df_relatorio.insert(0, 'Ranking', range(1, 1 + len(df_relatorio))) # Insere a coluna Ranking na posição 0

# 3. Renomear e Formatar as Colunas para o relatório
df_relatorio = df_relatorio.rename(columns={
    'NAME': 'Município',
    'HSP_CORRIGIDO_H/DIA': 'HSP (Horas/Dia)',
    'X_D_Declividade_Media': 'Declividade Média (°)',
    'X_S_Fator_Sombreamento': 'Fator Sombreamento (0-1)',
    'Fator_Capacidade_Est': 'Fator Capacidade Est. (%)',
    'Score_Final_Sitio': 'Score Final Sítio (0-100)'
})

# Formatação final para legibilidade
df_relatorio = df_relatorio.round({
    'HSP (Horas/Dia)': 2,
    'Declividade Média (°)': 2,
    'Fator Sombreamento (0-1)': 3,
    'Fator Capacidade Est. (%)': 2,
    'Score Final Sítio (0-100)': 1
})

# 4. EXPORTAR PARA CSV/EXCEL
nome_arquivo = 'Relatorio_Potencial_Solar_PR_Fase1.csv'
df_relatorio.to_csv(nome_arquivo, index=False, encoding='utf-8')

print(f"\n✅ Relatório final de 399 municípios gerado com sucesso!")
print(f"Arquivo salvo como: {nome_arquivo}")

# Exibe o Top 5 no console
print("\n--- TOP 5 MUNICÍPIOS ---")
print(df_relatorio.head().to_markdown(index=False))


✅ Relatório final de 399 municípios gerado com sucesso!
Arquivo salvo como: Relatorio_Potencial_Solar_PR_Fase1.csv

--- TOP 5 MUNICÍPIOS ---
|   Ranking | Município              |   HSP (Horas/Dia) |   Declividade Média (°) |   Fator Sombreamento (0-1) |   Fator Capacidade Est. (%) |   Score Final Sítio (0-100) |
|----------:|:-----------------------|------------------:|------------------------:|---------------------------:|----------------------------:|----------------------------:|
|         1 | Jardim Olinda          |              5.04 |                    2.41 |                      0.288 |                       16.81 |                        85.5 |
|         2 | Santo Antônio do Caiuá |              5.04 |                    2.97 |                      0.288 |                       16.79 |                        84.9 |
|         3 | Diamante do Norte      |              5.04 |                    2.88 |                      0.288 |                       16.8  |                   